* A formula of search is applied to OpenAlex to extract an equivalent database
* We load the inspire papers and extract the dois from the `recid`s
* We extract of the dois from the results in OpenAlex and compare with the dois from inspire:
  * `DD`: inspire dois
  * `FF`: OpenAlex dois
  * `FF-DD`: dois only in OpenAlex: see [Status analysis](#Status-analysis): `status` values are:
     * valid arXiv dois without extra dois: with `status['status'] = 'ok'` and `published = False`
     * new valid dois with `status['status'] = 'ok'` with `status['not_affiliations_in_inspire'] = True, ... 'mathching_inspire_authors', 'check_countries']`. The first fixes missing affiliations, the second are not in inspire but have inspire authors, the third may have errors in affiliations
     * duplicated entries in OpenAlex with `status in ['duplicated', 'duplicated_dois']`
     * erros in OpenAlex with `status in [wrong_countries, wrong_doi]`. The first are bad affilitions, the second is not the real doi 
     * extra works that maybe are outside the scope of inspire with `status = 'not_inspire_scope'`
     * `status = 'erratum_or_arxiv_doi'` ?
  * `DD-FF`: dois inly in inspire:

In [ ]:
[{'status': 'right_countries', 'doi': '10.1063/1.2710571'},
 {'status': 'duplicated_dois',
  'new_doi': '10.1007/s100520050258',
  'doi': '10.1007/s100529800817',
  'dois': [{'value': '10.1007/s100520050258'},
   {'source': 'ADS', 'value': '10.1007/s100529800817'}]},
 {'status': 'not_doi', 'recid': 1318327, 'doi': '10.48550/arxiv.1409.5695'},
 {'status': 'right_countries', 'doi': '10.1093/mnras/stu2558'},
 {'status': 'not_inspire_scope', 'doi': '10.24275/uami.gq67jr29s'},
 {'status': 'not_inspire_scope', 'doi': '10.2172/434870'},
 {'status': 'wrong_countries', 'doi': '10.1007/bf02817915'},
 {'status': 'duplicated_dois',
  'new_doi': '10.1016/0550-3213(95)00259-u',
  'doi': '10.1016/0550-3213(95)00620-6',
  'dois': [{'value': '10.1016/0550-3213(95)00259-U'},
   {'source': 'bibmatch', 'value': '10.1016/0550-3213(95)00620-6'}]},
 {'status': 'right_countries', 'doi': '10.1093/mnras/sty733'},
 {'status': 'duplicated',
  'arXiv': '10.48550/arxiv.2007.05004',
  'doi': '10.1103/physrevd.107.095030'}]

In [4]:
import requests
import json

In [2]:
LA = []
next_cursor = "*"
p = 200
url_topics = f"https://api.openalex.org/works?page=1&filter=topics.id:t10048|t10224|t10527|t10921|t11090|t10025|t10095|t11415|t11044|t11367|t10463,authors_count:1-10,authorships.countries:countries/co|countries/br|countries/ar|countries/cl|countries/mx|countries/pe|countries/ec|countries/uy|countries/bo|countries/cr|countries/ve|countries/cu|countries/do|countries/sv|countries/gt|countries/hn|countries/pa|countries/py,publication_year:1900-2022&sort=cited_by_count:desc&per_page={p}&cursor={next_cursor}"

In [3]:
loop = False
while loop:
    #url_topics = f"https://api.openalex.org/works?page=1&filter=authorships.author.id:a5049648126&sort=cited_by_count:desc&per_page=200&cursor={next_cursor}"
    
    j = requests.get(url_topics)
    
    la = j.json()
    
    la_meta = la.get('meta')
    next_cursor = la_meta.get('next_cursor')
    
    if not next_cursor:
        #break
        print('stop')
        break
    
    LA = LA + la.get('results')
    print(len(LA),end='\r')

In [7]:
if loop:
    with open("openalex_topics.json", "w") as json_file:
        json.dump(LA, json_file)
    print(len(LA))

In [378]:
with open('openalex_topics.json', 'r') as file:
        LA = json.load(file)

In [88]:
f = 'data/papers.json'
try:
    with open(f, 'r') as file:
        data = json.load(file)
        print("File data =", f)
except FileNotFoundError:
    print("Error: The file 'data.json' was not found.")
except json.JSONDecodeError:
    print("Error: Failed to decode JSON from the file.")

File data = data/papers.json


In [89]:
import pandas as pd

In [90]:
inspa = pd.DataFrame(data)

In [91]:
del data

In [13]:
import time

In [219]:
recids =inspa['recid'].unique()
len(recids)

49181

In [220]:
393487 in recids

True

In [221]:
#list(recids).index(393487)

In [208]:
#recids = recids[32287:32288]

In [222]:
loop = False
if loop:
    dois = []
    for x in list(range(0,len(L)+1,80))[1:]:
        time.sleep(0.1)
        print(f"{x-80}:{x}",end='\r')
    
        q='%20or%20'.join([ f"recid:{x}" for x in recids[x-80:x]])
        
        url = f"https://inspirehep.net/api//literature?size=100&page=1&q={q}&fields=dois"
        
        jj = requests.get(url)
        
        try:
            LL = jj.json().get('hits').get('hits')
        except:
            LL = []
        
        if not LL:
            continue
            
        tmp = pd.DataFrame(LL)
        tmp['control_number'] = tmp['metadata'].str['control_number']
        tmp['doi'] = tmp['metadata'].str['dois'].str[0].str['value']
        dois = dois + tmp[['control_number','doi']].dropna().to_dict(orient='records')
        #break

In [17]:
if loop:
    with open("inspire_dois.json", "w") as json_file:
        json.dump(dois, json_file)

In [223]:
del inspa

In [224]:
with open("inspire_dois.json", "r") as json_file:
    dois = json.load(json_file)

In [225]:
di = pd.DataFrame(dois)

In [227]:
di['doi'] = di['doi'].str.lower()

In [228]:
di['doi'].shape[0], di['doi'].unique().shape

(38713, (38713,))

In [379]:
df = pd.DataFrame(LA)

In [380]:
del LA

In [381]:
df.shape

(41731, 50)

Primary topics large that study area topics (11)

In [382]:
len(df['primary_topic'].str['display_name'].unique())

274

In [383]:
topics = url_topics.split('topics.id:')[-1].split(',')[0].split('|')
topics = [t.lower() for t in topics]

In [384]:
def topics_score(L,topics,score = 0):
    LL = []
    for d in L:
        if d['id'].split('/')[-1].lower() in topics and d['score'] > score:
            LL.append(d)
    return LL

In [385]:
df['filtered_topics'] = df['topics'].apply(lambda L: topics_score(L,topics,score=0))

In [386]:
df.shape

(41731, 51)

In [387]:
score = 0.9
df = df[df['topics'].apply(lambda L: topics_score(L,topics,score=score)).apply(len)> 0].reset_index(drop=True)
df.shape

(41347, 51)

In [388]:
df['doi'] = df['doi'].str.lower().str.split('https://doi.org/').str[-1]
df['doi'].shape,df['doi'].dropna().shape,df['doi'].dropna().unique().shape

((41347,), (38995,), (38993,))

In [401]:
dfa = df[df['doi'].apply(lambda d: d in intersection)].reset_index(drop=True)
authors = dfa['authorships'].apply(lambda L: [d.get('author').get('id').split('/')[-1] for d in L] )
authors = list(set([item for sublist in authors for item in sublist]))
len(authors) #authors both in inspire and OpenAlex

15527

In [34]:
DD = di['doi'].dropna().to_list() # dois in inspire (missing dois if duplicates)
FF = df['doi'].dropna().to_list() # dois in OpenAlex

In [36]:
len(DD)

38713

In [37]:
len(FF)

38995

In [38]:
intersection = list(set(DD).intersection( FF ))
len(intersection) # In both inspire and OpenAlex

27138

In [39]:
DDD = list(set(DD).difference( FF  ))
FFF = list(set(FF).difference(DD))

In [40]:
len(DDD) # only inspire

11575

In [131]:
with open('DDD.json','w') as f:
    json.dump(DDD,f)

In [41]:
len(FFF) # only OpenAlex

11855

In [43]:
df[df['doi'].astype(str).str.contains('arxiv')].shape # OpenAlex works in arXiv only 

(1277, 51)

In [44]:
trueFF = df[df['primary_location'].str['source'].str['issn_l'].apply(lambda s: s is not None)]['doi'].dropna().to_list()

In [45]:
len(trueFF),len(FF)

(35136, 38995)

In [46]:
trueFFF = list(set(trueFF).difference(DD))

In [47]:
len(trueFFF)

8941

In [61]:
del df

In [62]:
del dfa

In [433]:
def get_inspire(url):
    jj = requests.get(url)
        
    try:
        LL = jj.json().get('hits').get('hits')
    except:
        LL = []

    return LL
    
def clean_dois(dois):
    return [d for d in dois if d.get('source') != 'arXiv' and d.get('material') != 'erratum']


def get_arxiv(doi,recids):
    status = {}
    oadoi = None
    q = '/'.join(doi.split('/')[1:]).replace('arxiv.','arxiv:')
    url = f"https://inspirehep.net/api//literature?size=100&page=1&q={q}&fields=control_number,dois"
    LL = get_inspire(url)
    time.sleep(0.1)
    
    if LL:
        try:
            dois = clean_dois(LL[0].get('metadata').get('dois'))
            oadoi = dois[0].get('value').lower()
        except:
            oadoi = None
    
    if oadoi:
        url = f"https://api.openalex.org/works?filter=doi:{oadoi}"
        jj = requests.get(url)
        time.sleep(0.1)
    
        if jj.status_code == 200 and jj.json().get('meta') and jj.json().get('meta').get('count'):
            status = {'status':'duplicated',
             'arXiv': doi,
             'doi': oadoi
            }
            return status

    if LL:
        recid = LL[0].get('metadata').get('control_number')
        if recid in recids:
            status = {'status': 'not_doi','recid': recid, 'doi': doi}
        else:
            status = {'status': 'check_countries', 'doi': doi} # arXiv without doi

    return status

def get_dois(doi,DD):
    status = {}
    url = f"https://inspirehep.net/api/literature?q=doi:{doi}&fields=control_number,authors,dois,publication_info"
    LL = get_inspire(url)
    time.sleep(0.1)
    
    if LL and LL[0].get('metadata').get('dois'):
        dois = LL[0].get('metadata').get('dois')
        values_doi = [d.get('value') for d in dois]
        if doi in values_doi and len(values_doi) > 1:
            all_dois = LL[0].get('metadata').get('dois')
            dois = clean_dois(all_dois) # Clean unwanted dois
            new_doi = [d.get('value').lower() for d in dois if d.get('value') != doi]
            if new_doi:
                new_doi = new_doi[0]
                if new_doi in DD:
                    status = {'status': 'duplicated_dois', 'new_doi': new_doi, 'doi': doi, 'dois': all_dois}
                else:
                    status = {'status': 'wrong_doi','new_doi': new_doi, 'doi': doi, 'dois': all_dois}
            else:
                recid = LL[0].get('metadata').get('control_number')
                publication_info = LL[0].get('metadata').get('publication_info')
                in_recids = False
                if recid in recids:
                    in_recids = True    
                # bad doi was erratum or arXiv                
                status = {'status': 'erratum_or_arxiv_doi_or_out_of_range', 
                          'doi': doi, 
                          'dois': all_dois,
                          'recid': recid,
                          'in_recids': in_recids,
                          'publication_info': publication_info}

    return status

def get_countries(doi):
    status = {}
    url = f"https://inspirehep.net/api/literature?q=doi:{doi}&fields=authors,dois"
    LL = get_inspire(url)
    time.sleep(0.1)

    if LL:
        try:
            affiliations = [d for d in LL[0].get('metadata').get('authors') if d.get('affiliations')]
        except:
            affiliations = []
        
        if affiliations:
            status = {'status': 'wrong_countries', 'doi': doi} # right in inspirehep
        else:
            status = {'status': 'right_countries', 'doi': doi} # missing in inspirehep
            
    return status

def get_authors(doi,authors):
    status = {}
    url = f"https://api.openalex.org/works?filter=doi:{doi}"
    jj = requests.get(url)
    time.sleep(0.1)
    if jj.status_code == 200 and jj.json().get('meta') and jj.json().get('meta').get('count'):
        try:
            au = [ d.get('author').get('id').split('/')[-1] for d in jj.json().get('results')[0].get('authorships')]
        except:
            au = []
        auint = list(set(au).intersection(authors))    
        if len(auint) > 0:
            status = { 'status': 'ok',
                       'mathching_inspire_authors': True,
                       'mathching_inspire_author_ids': auint,
                       'doi': doi}
        else: # outside inspire
            status = { 'status': 'not_inspire_scope', 
                       'mathching_inspire_authors': False,
                       'mathching_inspire_author_ids': [],
                       'doi': doi}
    return status

In [286]:
no_issn_dois = list(set(FFF).difference(trueFFF)) # doi not relate to journals with issns
no_issn_dois[0:15]

['10.48550/arxiv.1909.10299',
 '10.48550/arxiv.1409.5695',
 '10.4171/90-1/4',
 '10.24275/uami.gq67jr29s',
 '10.2172/434870',
 '10.48550/arxiv.1603.03821',
 '10.22323/1.358.0411',
 '10.5281/zenodo.4593330',
 '10.48550/arxiv.2007.05004',
 '10.48550/arxiv.gr-qc/9904004',
 '10.48550/arxiv.1501.00331',
 '10.1017/cbo9780511617676.017',
 '10.1142/9789812811653_0017',
 '10.1109/ijcnn.2000.857891',
 '10.1007/978-3-0348-6201-1']

In [65]:
trueFFF[10:15] # Unknow affiliations in INSPIRE or missing in INSPIRE

['10.1016/j.astropartphys.2015.08.002',
 '10.1103/physrevd.93.124020',
 '10.1088/1742-6596/590/1/012008',
 '10.1093/mnras/stw1269',
 '10.1007/bf00762333']

In [240]:
FFF[10:20]

['10.1016/0003-4916(82)90173-7',
 '10.1140/epjc/s10052-022-11127-w',
 '10.1109/tasc.2022.3157659',
 '10.1140/epjc/s10052-015-3288-x',
 '10.1016/j.astropartphys.2015.08.002',
 '10.1109/pac.2005.1591321',
 '10.1103/physrevd.93.124020',
 '10.1088/1742-6596/590/1/012008',
 '10.1093/mnras/stw1269',
 '10.1007/bf00762333']

## Status analysis

In [245]:
2151035 not in recids

True

In [499]:
status['arxiv_doi'] = '10.48550/arxiv.0801.2559'
#status['arxiv_doi'] = '10.48550/arxiv.astro-ph/9505300'
def get_year(arxiv_doi):
    arxiv_id =arxiv_doi.split('/')
    if len(arxiv_id) == 2:
        year = eval('20'+arxiv_id[-1].split('arxiv.')[-1][:2])
    elif len(arxiv_id) == 3:
        year = arxiv_id[-1][:2]
        if year[0] == '0':
             year = eval('20' + year)
        else:
            year = eval('19' + year)
    
    return year

In [503]:
max_year = 2022
arxiv_doi = '10.48550/arxiv.0801.2559'
arxiv_doi = '10.48550/arxiv.astro-ph/9505300'
arxiv_doi = '10.48550/arxiv.astro-ph/0505300'
get_year(arxiv_doi)

2005

In [519]:
doi = '10.1088/1742-6596/590/1/012008'
doi =  '10.48550/arxiv.0801.2559'
#doi = '10.1103/physrevd.93.124020'
#doi = '10.48550/arxiv.astro-ph/0505300' # test: not_doi
#doi = '10.48550/arxiv.1909.10299' # test: duplicated
#doi = '10.1007/s100520050258' # test: duplicated_dois
#doi = '10.1140/epjc/s10052-015-3288-x' # test: right_countries
#doi = '10.1007/bf02817915' # test: wrong_countries
doi = '10.24275/uami.gq67jr29s' # test: 'mathching_inspire_authors' and 'status': 'not_inspire_scope'
doi = '10.1016/0003-4916(82)90173-7' # test matching_authors
doi = '10.1063/1.2710571' # test wrong_affiliations
doi = '10.1016/0550-3213(95)00620-6' # test duplicated dois
doi = '10.1088/1742-6596/590/1/012008' #test
doi = '10.1140/epjc/s10052-022-11127-w' # test outside_range
doi = '10.1007/bf00762333' # test 'mathching_inspire_authors'

In [541]:
def check_doi(doi, recids, DD, authors, max_year):
    url = f"https://inspirehep.net/api/literature?q=doi:{doi}&fields=control_number,authors,dois,publication_info"
    LL = get_inspire(url)
    time.sleep(0.1)

    
    status = {'doi':doi,'in_inspire': False}
    arxiv_doi = None
    if doi.find('/arxiv.') == 8:
        arxiv_doi = doi
    
    status['arxiv_doi'] = arxiv_doi
    
    metadata = None
    if LL:
        metadata = LL[0].get('metadata')
        status['in_inspire'] = True
    inspire_authors = None
    all_dois = None
    control_number = None
    publication_info = None
    if metadata:
        inspire_authors = metadata.get('authors')
        all_dois = metadata.get('dois')
        control_number = metadata.get('control_number')
        publication_info = metadata.get('publication_info')
    
    inspire_dois = []
    status['recid'] = control_number
    status['in_recids'] = False
    status['published'] = True
    status['erratum_doi'] = None
    status['inspire_dois'] = []
    status['new_doi'] = None
    status['oa_errors'] = []
    status['inspire_scope'] = True
    status['new_doi_in_inspire'] = None
    status['affiliations_in_inspire'] = None
    status['mathching_inspire_authors'] = None
    status['mathching_inspire_author_ids'] = None
    status['date_range'] = None
    status['year'] = None
    
    if publication_info:
        try:
            status['year'] = max([d.get('year') for d in publication_info])
        except:
            status['year'] = None
    
    if not status['year'] and status['arxiv_doi']:
        status['year'] = get_year(status['arxiv_doi'])
    
    # chek if already in database
    if control_number and control_number in recids:
        status['in_recids'] = True
    else:
        # check inspire_scope
        pass
    
    # Check if published with doi
    if all_dois:
        erratum_doi = [d for d in all_dois if d.get('material') == 'erratum']
        if erratum_doi:
            status['erratum_doi'] = erratum_doi[0]
        inspire_dois = [d for d in all_dois if d.get('source') != 'arXiv' and d.get('material') != 'erratum']
        status['inspire_dois'] = list(set([d.get('value').lower() for d in inspire_dois]))
    else:
        if status['in_inspire']:
            status['published'] = False
    
    # Analysis
    status['status'] = None
    if status.get('arxiv_doi') and status['inspire_dois']:
        status['new_doi'] = status['inspire_dois'][0]
        status['status'] = 'duplicated'
        status['oa_errors'].append(status['status'])
    else:
        status['status'] = 'ok'
    
    if not status.get('arxiv_doi') and status['inspire_dois']:
        if len(status['inspire_dois']) > 1:
            status['new_doi'] = [x for x in status['inspire_dois'] if x != doi ][0]
            status['status'] = 'duplicated_dois'
            status['oa_errors'].append(status['status'])
    else:
        if not status['oa_errors']:
            status['status'] = 'ok'
    
    if status['new_doi'] and status['new_doi'] in DD:
        status['new_doi_in_inspire'] = True
    else:
        status['new_doi_in_inspire'] = False
    if inspire_authors:
        affiliations = [d for d in inspire_authors if d.get('affiliations')]
        if affiliations: # not affiliations in inspire
            status['affiliations_in_inspire'] = True
        else:
            status['affiliations_in_inspire'] = False
    
    # not in inspire LA database but in inspire → affiliations exist and are not LA
    if status['in_inspire'] and not status['in_recids'] and status['affiliations_in_inspire']:
        # check year conflict
        if status['year'] and status['year'] >= max_year:
            status['status'] = 'ok'
            status['date_range'] = 'outside'
        else:
            status['status'] = 'wrong_countries'
            status['oa_errors'].append(status['status'])
    else:
        if not status['oa_errors']:
            status['status'] = 'ok'
    
    # not in inspire → check for inspire authors
    if not status['in_inspire'] and not status['in_recids']:
        check_status = get_authors(doi,authors)
        status['status'] = check_status['status']
        if status['status'] == 'not_inspire_scope':
            status['oa_errors'].append(status['status'])
        else:
            if not status['oa_errors']:
                status['status'] = 'ok'
            
        status['mathching_inspire_authors'] = check_status['mathching_inspire_authors']
        status['mathching_inspire_author_ids'] = check_status['mathching_inspire_author_ids']
    
    
    return status

In [542]:
#status

In [543]:
#dfa[dfa['authorships'].apply(lambda L: [d.get('author').get('id').split('/')[-1] for d in L] ).apply(lambda L: 'A5058759801' in L)]

In [544]:
#check_doi(doi,recids, DD, authors, max_year)

In [545]:
len(FFF)

11855

In [ ]:
ST = []
i = 0
for doi in FFF:
    status = {}
    i += 1
    print(str(i).zfill(5),end='\r',flush=True)
    status = check_doi(doi,recids, DD, authors, max_year)
    ST.append(status)

In [537]:
doi

'10.1088/1475-7516/2020/01/055'

In [518]:
ST

[{'doi': '10.1063/1.2710571',
  'in_inspire': True,
  'arxiv_doi': None,
  'recid': 1718980,
  'in_recids': False,
  'published': True,
  'erratum_doi': None,
  'inspire_dois': ['10.1063/1.2710571'],
  'new_doi': None,
  'oa_errors': [],
  'inspire_scope': True,
  'new_doi_in_inspire': False,
  'affiliations_in_inspire': False,
  'mathching_inspire_authors': None,
  'mathching_inspire_author_ids': None,
  'date_range': None,
  'year': 2007,
  'status': 'ok'},
 {'doi': '10.1007/s100529800817',
  'in_inspire': True,
  'arxiv_doi': None,
  'recid': 451859,
  'in_recids': True,
  'published': True,
  'erratum_doi': None,
  'inspire_dois': ['10.1007/s100529800817', '10.1007/s100520050258'],
  'new_doi': '10.1007/s100520050258',
  'oa_errors': ['duplicated_dois'],
  'inspire_scope': True,
  'new_doi_in_inspire': True,
  'affiliations_in_inspire': True,
  'mathching_inspire_authors': None,
  'mathching_inspire_author_ids': None,
  'date_range': None,
  'year': 1998,
  'status': 'duplicated_do

In [390]:
ST = []
i = 0
for doi in FFF[0:20]:
    status = {}
    i += 1
    print(str(i).zfill(5),end='\r',flush=True)

    #status = get_recid(doi,recids)
    #if status:
    #    ST.append(status)
    #    #pass
    #    continue
    
    
    if doi.find('/arxiv.') == 8:
        status = get_arxiv(doi,recids)
    if status:
        ST.append(status)
        #pass
        continue

    status = get_countries(doi)
    if status:
        ST.append(status)
        #pass
        continue

    
    status = get_dois(doi,DD)
    if status:
        ST.append(status)
        #pass
        continue

    status = get_authors(doi,authors)
    ST.append(status)
    #if i%100 == 0:


In [367]:
doi

'10.24275/uami.gq67jr29s'

In [391]:
ST

[{'status': 'right_countries', 'doi': '10.1063/1.2710571'},
 {'status': 'wrong_countries', 'doi': '10.1007/s100529800817'},
 {'status': 'not_doi', 'recid': 1318327, 'doi': '10.48550/arxiv.1409.5695'},
 {'status': 'right_countries', 'doi': '10.1093/mnras/stu2558'},
 {'status': 'not_inspire_scope', 'doi': '10.24275/uami.gq67jr29s'},
 {'status': 'not_inspire_scope', 'doi': '10.2172/434870'},
 {'status': 'wrong_countries', 'doi': '10.1007/bf02817915'},
 {'status': 'wrong_countries', 'doi': '10.1016/0550-3213(95)00620-6'},
 {'status': 'right_countries', 'doi': '10.1093/mnras/sty733'},
 {'status': 'duplicated',
  'arXiv': '10.48550/arxiv.2007.05004',
  'doi': '10.1103/physrevd.107.095030'},
 {'status': 'mathching_inspire_authors',
  'authors_id': ['A5053412812', 'A5068807109'],
  'doi': '10.1016/0003-4916(82)90173-7'},
 {'status': 'wrong_countries', 'doi': '10.1140/epjc/s10052-022-11127-w'},
 {'status': 'not_inspire_scope', 'doi': '10.1109/tasc.2022.3157659'},
 {'status': 'right_countries', 

In [258]:
1331731 in recids

False

In [183]:
'10.1016/0550-3213(95)00620-6' in DD

False

In [217]:
'10.1016/0550-3213(95)00259-u' in DD

True

In [116]:
doi = '10.48550/arxiv.1409.5695'

In [127]:
    status = {}
    oadoi = None
    q = '/'.join(doi.split('/')[1:]).replace('arxiv.','arxiv:')
    url = f"https://inspirehep.net/api//literature?size=100&page=1&q={q}&fields=control_number,dois"
    LL = get_inspire(url)
    time.sleep(0.1)
    
    if LL:
        try:
            oadoi = [ d for d in LL[0].get('metadata').get('dois') if d.get('source') != 'arXiv' and d.get('material') != 'erratum' 
              ][0].get('value').lower()
        except:
            oadoi = None
    
    if oadoi:
        url = f"https://api.openalex.org/works?filter=doi:{oadoi}"
        jj = requests.get(url)
        time.sleep(0.1)
    
        if jj.status_code == 200 and jj.json().get('meta') and jj.json().get('meta').get('count'):
            status = {'status':'duplicated',
             'arXiv': doi,
             'doi': oadoi
            }
            #return status

    recid = LL[0].get('metadata').get('control_number')
    if LL:
        if recid in recids:
            status = {'status': 'not_doi','recid': recid, 'doi': doi}
        else:
            status = {'status': 'check_countries', 'doi': doi} # arXiv without doi

In [128]:
status

{'status': 'not_doi', 'recid': 1318327, 'doi': '10.48550/arxiv.1409.5695'}

In [120]:
LL[0].get('metadata').get('control_number')

1318327

In [87]:
ST

[{'status': 'right_countries', 'doi': '10.1063/1.2710571'},
 {'status': 'wrong_countries', 'doi': '10.1007/s100529800817'},
 {'status': 'check_countries', 'doi': '10.48550/arxiv.1409.5695'},
 {'status': 'right_countries', 'doi': '10.1093/mnras/stu2558'},
 {'status': 'not_mathching_authors', 'doi': '10.24275/uami.gq67jr29s'},
 {'status': 'not_mathching_authors', 'doi': '10.2172/434870'},
 {'status': 'wrong_countries', 'doi': '10.1007/bf02817915'},
 {'status': 'wrong_countries', 'doi': '10.1016/0550-3213(95)00620-6'},
 {'status': 'right_countries', 'doi': '10.1093/mnras/sty733'},
 {'status': 'duplicated',
  'arXiv': '10.48550/arxiv.2007.05004',
  'doi': '10.1103/physrevd.107.095030'}]

In [83]:
get_countries(doi)

{'status': 'wrong_countries', 'doi': '10.1007/s100529800817'}

In [73]:
i += 1
print(str(i).zfill(5),end='\r')
 